# LINGUAGEM NATURAL E INTELIGÊNCIA ARTIFICIAL


## CLASSIFICAÇÃO AUTOMÁTICA DE TEXTO

#### Exercício:

Os dados utilizados neste exemplo são dois ficheiros em formato CSV:

#### Livrarias python

In [45]:
import sys
from openpyxl import Workbook
import csv
import re
import pickle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#### Livrarias NLTK - Natural Langage Processing Toolkit

In [46]:
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

#### Livrarias SciKit Learn - Machine Learning

In [47]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

#### Variáveis

In [48]:
corpo = []
novo_corpo = []
categorias = [
        "Outra",
        "Astronomia",
        "Biologia",
        "Geologia",
        "Farois",
        "Engenharia",
        "Patrimonio",
        "Castelos",
    ]

#### Funções

In [49]:
def tokenizar(texto, stemizar=True, lingua="portuguese"):

    # Usa o SnowballStemmer do NLTK para português
    stemizador = SnowballStemmer(lingua)

    # Tokeniza por frase e por palavra
    tokens = [word.lower() for frase in nltk.sent_tokenize(texto) for word in nltk.word_tokenize(frase)]
    
    # Aplica NLTK stopwords 
    stop_words_pt = set(stopwords.words(lingua)) 
    
    # Filtra as palavras que não contenham letras (e.g., numbers and raw punctuation) 
    # e que estejam na lista de stopwords
    tokens_filtrados = []
    for token in tokens:
        if re.search("[a-zA-Z]", token):
            if token not in stopwords.words(lingua):
                if token not in stop_words_pt:
                    tokens_filtrados.append(token)
                if stemizar:
                    stems = [stemizador.stem(token) for token in tokens_filtrados]
    return stems


In [21]:
def _convert_to_number(cell):
    if cell.isnumeric():
        return int(cell)
    try:
        return float(cell)
    except ValueError:
        return cell

#### Dados para treino

In [50]:
# Importando os dados de treino
dados_treino = pd.read_csv("data/dados_treino.csv", delimiter = "\t", quoting = 3)

#### Pré-processamento dos dados de treino

In [51]:
print("\nTreinando com o corpo de dados já classificado ... Aguardar, por favor.\n")
for i in range(len(dados_treino)):    
    tokens = tokenizar(dados_treino["texto"][i], stemizar=True, lingua="portuguese")    
    # refaz cada linha com os tokens processados
    tokens = ' '.join(tokens)
    corpo.append(tokens)


Treinando com o corpo de dados já classificado ... Aguardar, por favor.



#### Calcular a matriz TF-IDF

In [52]:
# Filtrar só as 100 palavras mais frequentes
cv = CountVectorizer(max_features = 100)

# Obter a variável independente X (o texto classificado)
X = cv.fit_transform(corpo).toarray()
tfidf_transformer = TfidfTransformer()
X_tfidf = tfidf_transformer.fit_transform(X)

# Gravar a matriz TF-IDF
pickle.dump(X_tfidf, open("TF-IDF_Matrix.p", "wb"))

# Obter a variável independente y (a categoria a que o texto pertence)
y = dados_treino.iloc[:, 2].values # ":" para ter todas as linhas, e "2" para a coluna

#### Aplicar o modelo classificação aos dados de treino

In [25]:
# Aplicar o modelo Multinominal Naive Bayes (Multinomial NB model),
# um método supervisionado e probabilístico de aprendizagem
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB().fit(X_tfidf, y)

#### Importar novos dados para classificação

In [26]:
# Importando os dados para classificação 
dados_classificar = pd.read_csv("data/dados_classificar.csv", delimiter = "\t", quoting = 3)

A classificar actividades, partindo do treino realizado ... Aguardar, por favor.



#### Pré-processamento dos dados para classificar

In [27]:
print("A classificar actividades, partindo do treino realizado ... Aguardar, por favor.\n")
for i in range(len(dados_classificar)):    
    tokens = tokenizar(dados_classificar["texto"][i], stemizar=True, lingua="portuguese")    
    # refaz cada linha com os tokens processados
    tokens = ' '.join(tokens)
    novo_corpo.append(tokens)

A classificar actividades, partindo do treino realizado ... Aguardar, por favor.



#### Calcular a matiz TF-IDF dos dados para classificar

In [28]:
X_novo = cv.transform(novo_corpo)
X_novo_tfidf = tfidf_transformer.transform(X_novo)

#### Previsão de classificações e respectivas probabilidades

In [ ]:
previsoes = clf.predict(X_novo_tfidf)
probabilidades = clf.predict_proba(X_novo_tfidf)

nova_classificacao = []
for i in range(len(dados_classificar)): 
    p = previsoes[i]
    if p > 5:
        p = 5
    pb = p - 1
    print(str(i) + "\t" + str(p) + "\t" + str(probabilidades[i][pb]*100))

    line = {
            "titulo":dados_classificar["titulo"][i],
            "texto":dados_classificar["texto"][i],
            "previsao":p,
            "categoria":categorias[previsoes[i]],
            "probabilidade":(probabilidades[i][pb]) * 100,             
            }
    nova_classificacao.append(line)


#### Escrever  os dados classificados num ficheiro CSV

In [33]:
with open("data/dados_classificados.csv", "w") as ficheiro_csv:       
    csv.writer(ficheiro_csv, delimiter = "\t", quotechar = ",", quoting=csv.QUOTE_MINIMAL)
    colunas = ["titulo","texto", "tipo", "previsao","probabilidade"]
    linha = {}
    writer = csv.DictWriter(ficheiro_csv, colunas, delimiter='\t')
    writer.writeheader()   
    

    for i in range(len(nova_classificacao)):
        linha = {
        "titulo": nova_classificacao[i]["titulo"],
        "texto": nova_classificacao[i]["texto"],
        "tipo": nova_classificacao[i]["previsao"],                
        "previsao": nova_classificacao[i]["categoria"],
        "probabilidade": nova_classificacao[i]["probabilidade"]                          
        }    
        writer.writerow(linha)

#### Converter o ficheiro CSV para Excel

In [ ]:
wb = Workbook()
worksheet = wb.active
fname = "data/dados_classificados.csv"
for row in csv.reader(open(fname), delimiter="\t"):
    worksheet.append([_convert_to_number(cell) for cell in row])
wb.save(fname.replace(".csv", ".xlsx"))
print("\nA gravar o resultado no ficheiro excel dados_classificados.xlsx.\n")
print("Terminado!\n")